LINKS TO DATASETS:
**weather**: https://www.kaggle.com/datasets/guillemservera/global-daily-climate-data

**flight delay**:https://www.kaggle.com/datasets/arvindnagaonkar/flight-delay?select=Flight_Delay.parquet


# Metadata for Flight Delays Dataset

## Metadata

| Column                   | Description                                                                    |
|--------------------------|--------------------------------------------------------------------------------|
| year                     | Year                                                                           |
| quarter                  | Quarter of the year                                                            |
| day_of_month             | Day of the month                                                               |
| day_of_week              | Day of the week                                                                |
| flight_date              | Date of the flight                                                             |
| marketing_airline_network| Unique Marketing Carrier Code                                                  |
| origin_city_name         | Origin Airport, City Name                                                      |
| dest_city_name           | Destination Airport, City Name                                                 |
| crs_dep_time             | Scheduled Departure Time (local time: hhmm)                                    |
| dep_time                 | Actual Departure Time (local time: hhmm)                                       |
| dep_delay                | Difference in minutes between scheduled and actual departure time               |
| dep_delay_minutes        | Difference in minutes between scheduled and actual departure time (early departures set to 0) |
| taxi_out                 | Taxi Out Time (duration from gate to runway) in Minutes                        |
| wheels_off               | Wheels-Off Time (local time: hhmm)                                             |
| wheels_on                | Wheels-On Time (local time: hhmm)                                              |
| taxi_in                  | Taxi In Time (duration from runway to gate) in Minutes                         |
| crs_arr_time             | Scheduled Arrival Time (local time: hhmm)                                      |
| arr_time                 | Actual Arrival Time (local time: hhmm)                                         |
| arr_delay                | Difference in minutes between scheduled and actual arrival time                 |
| arr_delay_minutes        | Difference in minutes between scheduled and actual arrival time (early arrivals set to 0) |
| crs_elapsed_time         | Scheduled Elapsed Time of Flight in Minutes                                    |
| actual_elapsed_time      | Elapsed Time of Flight in Minutes                                              |
| air_time                 | Flight Time in Minutes                                                         |
| distance                 | Distance between airports (miles)                                              |
| distance_group           | Distance Intervals (every 250 miles) for Flight Segment                        |
| carrier_delay            | Carrier Delay in Minutes                                                       |
| weather_delay            | Weather Delay in Minutes                                                       |
| nas_delay                | National Air System Delay in Minutes                                           |
| security_delay           | Security Delay in Minutes                                                      |

In [14]:
!pip install kagglehub
!pip install openpyxl



   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpy

In [4]:
import kagglehub
#weather_path = kagglehub.dataset_download("selfishgene/historical-hourly-weather-data")
weather_path = kagglehub.dataset_download("guillemservera/global-daily-climate-data")
flight_path = kagglehub.dataset_download("arvindnagaonkar/flight-delay")

In [5]:
print("Weather dataset path:", weather_path)
print("Flight delay dataset path:", flight_path)

Weather dataset path: C:\Users\steve\.cache\kagglehub\datasets\guillemservera\global-daily-climate-data\versions\20
Flight delay dataset path: C:\Users\steve\.cache\kagglehub\datasets\arvindnagaonkar\flight-delay\versions\2


In [6]:
#Load weather data (city attributes + example weather features)
import pandas as pd

city_df = pd.read_csv(f"{weather_path}/cities.csv")
countries_df = pd.read_csv(f"{weather_path}/countries.csv")
daily_weather_df = pd.read_parquet(f"{weather_path}/daily_weather.parquet")
#weatherDescription_df = pd.read_csv(f"{weather_path}/weather_description.csv")

In [7]:
#Load flight delay data (.parquet format).
# Select columns needed for integration and modeling (so that we dont use too much memory)
columns_needed = [
    'FlightDate', 'OriginCityName', 'DestCityName',
    'CRSDepTime', 'DepTime', 'DepDelayMinutes',
    'WeatherDelay', 'Distance', 'ArrDelayMinutes'
]

flight_df = pd.read_parquet(f"{flight_path}/Flight_Delay.parquet", columns=columns_needed)


In [8]:
#see what columns the datasets have
print("City attributes columns:", city_df.columns)
print("Countries columns:", countries_df.columns)
print("Weather Description columns:",daily_weather_df.columns)
print("Flight delay columns:", flight_df.columns)
print(flight_df.head())

City attributes columns: Index(['station_id', 'city_name', 'country', 'state', 'iso2', 'iso3',
       'latitude', 'longitude'],
      dtype='object')
Countries columns: Index(['country', 'native_name', 'iso2', 'iso3', 'population', 'area',
       'capital', 'capital_lat', 'capital_lng', 'region', 'continent'],
      dtype='object')
Weather Description columns: Index(['station_id', 'city_name', 'date', 'season', 'avg_temp_c', 'min_temp_c',
       'max_temp_c', 'precipitation_mm', 'snow_depth_mm', 'avg_wind_dir_deg',
       'avg_wind_speed_kmh', 'peak_wind_gust_kmh', 'avg_sea_level_pres_hpa',
       'sunshine_total_min'],
      dtype='object')
Flight delay columns: Index(['FlightDate', 'OriginCityName', 'DestCityName', 'CRSDepTime', 'DepTime',
       'DepDelayMinutes', 'WeatherDelay', 'Distance', 'ArrDelayMinutes'],
      dtype='object')
   FlightDate OriginCityName    DestCityName  CRSDepTime  DepTime  \
1  2018-01-15     Newark, NJ  Charleston, SC        1845   1928.0   
2  2018-01-16 

# Part C: Data Integration



In [9]:
#next step is to start cleaning and preparing the data

#clean city names for better matching
flight_df['OriginCityName_clean'] = flight_df['OriginCityName'].str.extract(r'^(.*?)(?:,|$)', expand=False).str.strip().str.lower()
daily_weather_df['city_name_clean'] = daily_weather_df['city_name'].str.strip().str.lower()

In [10]:
#convert Flight Date and Weather Date to Datetime

flight_df['FlightDate'] = pd.to_datetime(flight_df['FlightDate'])
daily_weather_df['date'] = pd.to_datetime(daily_weather_df['date'])

print(flight_df.head())
print(daily_weather_df.head())

  FlightDate OriginCityName    DestCityName  CRSDepTime  DepTime  \
1 2018-01-15     Newark, NJ  Charleston, SC        1845   1928.0   
2 2018-01-16     Newark, NJ  Charleston, SC        1835   1956.0   
3 2018-01-17     Newark, NJ  Charleston, SC        1835   1836.0   
4 2018-01-18     Newark, NJ  Charleston, SC        1845   1844.0   
6 2018-01-20     Newark, NJ  Charleston, SC        1835   1829.0   

   DepDelayMinutes  WeatherDelay  Distance  ArrDelayMinutes  \
1             43.0           0.0     628.0             41.0   
2             81.0           0.0     628.0             69.0   
3              1.0           0.0     628.0              0.0   
4              0.0           0.0     628.0              0.0   
6              0.0           0.0     628.0              0.0   

  OriginCityName_clean  
1               newark  
2               newark  
3               newark  
4               newark  
6               newark  
  station_id city_name       date  season  avg_temp_c  min_tem

In [ ]:
#merge flight data with weather data for origin cities
#match on city name and date
merged_df = flight_df.merge(
    daily_weather_df,
    left_on=['OriginCityName_clean', 'FlightDate'],
    right_on=['city_name_clean', 'date'],
    how='left',
    suffixes=('', '_origin_weather')
)

print(f"Original flight data shape: {flight_df.shape}")
print(f"After merging with weather data: {merged_df.shape}")
print(f"\nSuccessful weather matches: {merged_df['avg_temp_c'].notna().sum()} out of {len(merged_df)} flights")
print("\nSample of merged data:")
print(merged_df[['FlightDate', 'OriginCityName', 'avg_temp_c', 'precipitation_mm', 'avg_wind_speed_kmh', 'DepDelayMinutes']].head(10))

Original flight data shape: (30132672, 10)
After merging with weather data: (30250045, 25)

Successful weather matches: 7952755 out of 30250045 flights

Sample of merged data:
  FlightDate OriginCityName  avg_temp_c  precipitation_mm  avg_wind_speed_kmh  \
0 2018-01-15     Newark, NJ         NaN               NaN                 NaN   
1 2018-01-16     Newark, NJ         NaN               NaN                 NaN   
2 2018-01-17     Newark, NJ         NaN               NaN                 NaN   
3 2018-01-18     Newark, NJ         NaN               NaN                 NaN   
4 2018-01-20     Newark, NJ         NaN               NaN                 NaN   
5 2018-01-21     Newark, NJ         NaN               NaN                 NaN   
6 2018-01-22     Newark, NJ         NaN               NaN                 NaN   
7 2018-01-23     Newark, NJ         NaN               NaN                 NaN   
8 2018-01-25     Newark, NJ         NaN               NaN                 NaN   
9 2018-01-26  

In [16]:
#export orirginal merged df to excel
merged_df.to_csv('merged_data.csv', index=False)

In [ ]:
#check integration quality and coverage
print("=== Data Integration Statistics ===\n")

#count matched vs unmatched records
matched = merged_df['station_id'].notna().sum()
total = len(merged_df)
match_rate = (matched / total) * 100

print(f"Total flights: {total:,}")
print(f"Flights with weather data: {matched:,}")
print(f"Match rate: {match_rate:.2f}%")
print(f"Flights without weather data: {total - matched:,}\n")

#check which cities have the best coverage
city_coverage = merged_df.groupby('OriginCityName').agg({
    'station_id': lambda x: x.notna().sum(),
    'FlightDate': 'count'
}).rename(columns={'station_id': 'with_weather', 'FlightDate': 'total_flights'})
city_coverage['match_rate'] = (city_coverage['with_weather'] / city_coverage['total_flights'] * 100).round(2)
city_coverage = city_coverage.sort_values('match_rate', ascending=False)

print("Top 10 cities by weather data coverage:")
print(city_coverage.head(10))
print("\nBottom 10 cities by weather data coverage:")
print(city_coverage.tail(10))

=== Data Integration Statistics ===

Total flights: 30,250,045
Flights with weather data: 8,099,227
Match rate: 26.77%
Flights without weather data: 22,150,818

Top 10 cities by weather data coverage:
                 with_weather  total_flights  match_rate
OriginCityName                                          
Des Moines, IA          69088          69088       100.0
CONCORD, NC              1126           1126       100.0
Austin, TX             284749         284749       100.0
Harrisburg, PA          41820          41820       100.0
Columbia, MO             9675           9675       100.0
Santa Rosa, CA          15783          15783       100.0
Columbia, SC            39503          39503       100.0
Columbus, GA             6262           6262       100.0
Tallahassee, FL         24070          24070       100.0
Columbus, MS             4895           4895       100.0

Bottom 10 cities by weather data coverage:
                                with_weather  total_flights  match_rate

In [ ]:
#specific examples of integrated data
print("=== Examples of Data Integration Cases ===\n")

#Example 1: Successful integration
print("Example 1: Flights WITH weather data")
example_with_weather = merged_df[merged_df['avg_temp_c'].notna()].head(3)
print(example_with_weather[['FlightDate', 'OriginCityName', 'DepDelayMinutes', 
                             'avg_temp_c', 'precipitation_mm', 'avg_wind_speed_kmh']])

print("\n" + "="*70 + "\n")

#Example 2: Failed integration
print("Example 2: Flights WITHOUT weather data (no match found)")
example_without_weather = merged_df[merged_df['avg_temp_c'].isna()].head(3)
print(example_without_weather[['FlightDate', 'OriginCityName', 'DepDelayMinutes', 
                                'avg_temp_c', 'precipitation_mm']])

print("\n" + "="*70 + "\n")

#Example 3: High precipitation days
print("Example 3: Flights on high precipitation days (>10mm)")
high_precip = merged_df[merged_df['precipitation_mm'] > 10].head(3)
print(high_precip[['FlightDate', 'OriginCityName', 'DepDelayMinutes', 
                    'WeatherDelay', 'precipitation_mm', 'avg_wind_speed_kmh']])

=== Examples of Data Integration Cases ===

Example 1: Flights WITH weather data
    FlightDate  OriginCityName  DepDelayMinutes  avg_temp_c  precipitation_mm  \
15  2018-01-07  Providence, RI              0.0       -14.6               0.0   
299 2018-01-02  Des Moines, IA             27.0       -20.5               0.0   
300 2018-01-06  Des Moines, IA             10.0       -15.7               0.0   

     avg_wind_speed_kmh  
15                 15.1  
299                19.8  
300                15.8  


Example 2: Flights WITHOUT weather data (no match found)
  FlightDate OriginCityName  DepDelayMinutes  avg_temp_c  precipitation_mm
0 2018-01-15     Newark, NJ             43.0         NaN               NaN
1 2018-01-16     Newark, NJ             81.0         NaN               NaN
2 2018-01-17     Newark, NJ              1.0         NaN               NaN


Example 3: Flights on high precipitation days (>10mm)
    FlightDate OriginCityName  DepDelayMinutes  WeatherDelay  \
573 2018-01

In [ ]:
#Analyze functional dependencies
print("=== Functional Dependencies Analysis ===\n")

#1. FlightDate + OriginCity -> Weather conditions
print("Dependency 1: FlightDate + OriginCity → Weather Conditions")
print("If we know the date and origin city, we can determine weather conditions.")
sample_dep1 = merged_df[merged_df['avg_temp_c'].notna()].groupby(['FlightDate', 'OriginCityName_clean']).agg({
    'avg_temp_c': 'first',
    'precipitation_mm': 'first',
    'avg_wind_speed_kmh': 'first'
}).head(5)
print(sample_dep1)
print()

#2. Distance dependency
print("\nDependency 2: OriginCity + DestinationCity → Distance")
print("If we know origin and destination, distance is fixed.")
sample_dep2 = merged_df.groupby(['OriginCityName', 'DestCityName'])['Distance'].agg(['first', 'nunique', 'count'])
sample_dep2.columns = ['Distance', 'Unique_Values', 'Flight_Count']
print(sample_dep2.head())
print(f"Verification: Each route has {sample_dep2['Unique_Values'].max()} unique distance value (should be 1)")
print()

#3. Weather delay dependency
print("\nDependency 3: Relationship between Weather Conditions and Weather Delay")
print("Poor weather conditions may correlate with weather delays.")
weather_impact = merged_df[merged_df['precipitation_mm'].notna()].groupby(
    pd.cut(merged_df['precipitation_mm'], bins=[0, 1, 5, 10, 100], labels=['None/Light', 'Light', 'Moderate', 'Heavy'])
).agg({
    'WeatherDelay': 'mean',
    'DepDelayMinutes': 'mean',
    'FlightDate': 'count'
})
weather_impact.columns = ['Avg_Weather_Delay', 'Avg_Departure_Delay', 'Flight_Count']
print(weather_impact)

=== Functional Dependencies Analysis ===

Dependency 1: FlightDate + OriginCity → Weather Conditions
If we know the date and origin city, we can determine weather conditions.
                                 avg_temp_c  precipitation_mm  \
FlightDate OriginCityName_clean                                 
2018-01-01 albany                     -17.3               0.0   
           atlanta                     -4.1               0.0   
           austin                      -2.3               0.0   
           boise                       -3.3               0.0   
           boston                     -14.7               0.0   

                                 avg_wind_speed_kmh  
FlightDate OriginCityName_clean                      
2018-01-01 albany                               8.6  
           atlanta                             21.2  
           austin                              12.2  
           boise                                4.3  
           boston                            

C:\Users\steve\AppData\Local\Temp\ipykernel_31888\1652483931.py:27: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  weather_impact = merged_df[merged_df['precipitation_mm'].notna()].groupby(


                  Avg_Weather_Delay  Avg_Departure_Delay  Flight_Count
precipitation_mm                                                      
None/Light                 0.780456            15.139806        645152
Light                      1.247871            17.358439        691022
Moderate                   1.462657            18.703186        370212
Heavy                      2.879042            23.474611        604649


In [ ]:
#temp patterns
print("=== Temporal Integration Patterns ===\n")

#monthly coverage
merged_df['Month'] = merged_df['FlightDate'].dt.month
monthly_coverage = merged_df.groupby('Month').agg({
    'station_id': lambda x: (x.notna().sum() / len(x) * 100)
}).rename(columns={'station_id': 'Weather_Coverage_%'})
print("Weather data coverage by month:")
print(monthly_coverage.round(2))

print("\n" + "="*70 + "\n")

#correlation between weather and delay
print("Correlation between weather features and delays:")
weather_delay_cols = ['avg_temp_c', 'precipitation_mm', 'avg_wind_speed_kmh', 
                      'DepDelayMinutes', 'WeatherDelay', 'ArrDelayMinutes']
correlation_matrix = merged_df[weather_delay_cols].corr()
print(correlation_matrix[['DepDelayMinutes', 'WeatherDelay']].sort_values('WeatherDelay', ascending=False))

=== Temporal Integration Patterns ===

Weather data coverage by month:
       Weather_Coverage_%
Month                    
1                   26.77
2                   26.71
3                   26.73
4                   26.90
5                   26.61
6                   26.57
7                   26.64
8                   26.84
9                   26.95
10                  26.92
11                  26.91
12                  26.74


Correlation between weather features and delays:
                    DepDelayMinutes  WeatherDelay
WeatherDelay               0.306218      1.000000
ArrDelayMinutes            0.981775      0.313277
DepDelayMinutes            1.000000      0.306218
precipitation_mm           0.064809      0.048897
avg_wind_speed_kmh         0.027882      0.014568
avg_temp_c                -0.000038     -0.007116


# Part D: Data Preparation

## Handle Null Data - Identify Missing Values


In [ ]:
#analyze null
print("=== Null Data Analysis ===\n")

#missing value stats
null_stats = pd.DataFrame({
    'Column': merged_df.columns,
    'Null_Count': merged_df.isnull().sum().values,
    'Null_Percentage': (merged_df.isnull().sum().values / len(merged_df) * 100).round(2)
})
null_stats = null_stats[null_stats['Null_Count'] > 0].sort_values('Null_Percentage', ascending=False)

print("Columns with missing values:")
print(null_stats)
print()


key_columns = ['DepTime', 'DepDelayMinutes', 'WeatherDelay', 'avg_temp_c', 
               'precipitation_mm', 'avg_wind_speed_kmh', 'ArrDelayMinutes']
print("\nMissing data in key columns:")
for col in key_columns:
    if col in merged_df.columns:
        missing = merged_df[col].isnull().sum()
        total = len(merged_df)
        pct = (missing / total * 100)
        print(f"{col:25s}: {missing:7,} ({pct:5.2f}%)")

=== Null Data Analysis ===

Columns with missing values:
                    Column  Null_Count  Null_Percentage
23      sunshine_total_min    30248207            99.99
21      peak_wind_gust_kmh    30245664            99.99
18           snow_depth_mm    23284218            76.97
19        avg_wind_dir_deg    22981356            75.97
22  avg_sea_level_pres_hpa    22567007            74.60
20      avg_wind_speed_kmh    22332980            73.83
14              avg_temp_c    22297290            73.71
17        precipitation_mm    22215998            73.44
16              max_temp_c    22168627            73.28
15              min_temp_c    22154523            73.24
12                    date    22150818            73.23
13                  season    22150818            73.23
10              station_id    22150818            73.23
11               city_name    22150818            73.23
24         city_name_clean    22150818            73.23


Missing data in key columns:
DepTime         